In [2]:
import numpy as np
from typing import Optional, Tuple
from scipy.optimize import fsolve

In [77]:
class fuselage:
    def __init__(self) -> None:
        self.length: Optional[float] = 18
        self.diameter: Optional[float] = 2

        self.tailboom_length: Optional[float] = 9
        self.tailboom_diameter: Optional[float] = 0.7

        self.hrztail_h: Optional[float] = 1.5
        self.hrztail_b: Optional[float] = 0.5
        self.hrztail_t: Optional[float] = 0.3
    
    #downwash
    def Re_down(self) -> float:
        rho = 0.9
        V = 100
        # in the most simiplied way, take this as the lamba inflow for hover at that altitude
        Vin = 5
        velocity = V + Vin

        d = self.diameter
        nu = 0.00002

        return (rho*velocity*d)/nu

    def Downwash(self) -> float:
        Re = self.Re_down()
        cd = (0.455/(np.log10(Re))) - (1700/Re)
        Ki = 1.3
        SA = (np.pi)*self.diameter*self.length

        return SA*Ki*cd

    # backwards drag
    def Re_back_body(self):
        rho = 0.9
        velocity = 60
        d = self.length
        nu = 0.00002

        return (rho*velocity*d)/nu
    
    def D1(self) -> float:
        Re = self.Re_back_body()
        cd = (0.455/(np.log10(Re))) - (1700/Re)
        SA = (np.pi)*self.diameter*self.length

        return SA*cd
    
    def Re_back_tail(self):
        rho = 0.9
        velocity = 60
        d = self.tailboom_length
        nu = 0.00002

        return (rho*velocity*d)/nu
    
    def D2(self) -> float:
        Re = self.Re_back_tail()
        cd = (0.455/(np.log10(Re))) - (1700/Re)
        SA = (np.pi)*self.tailboom_diameter*self.length

        return SA*cd
    
    def Re_back_hrztail(self):
        rho = 0.9
        velocity = 60
        d = self.hrztail_b
        nu = 0.00002

        return (rho*velocity*d)/nu
    
    def D3(self) -> float:
        Re = self.Re_back_hrztail()
        cd = (0.455/(np.log10(Re))) - (1700/Re)
        SA = (np.pi)*self.hrztail_b*self.length

        return SA*cd
    
    def output(self):
        return self.D1() + self.D2() + self.D3()

In [78]:
drag = fuselage()
drag.output()

np.float64(11.189493432941742)

TRIM CONDITIONS:

In [28]:
class Rotor:
    def __init__(self) -> None:
        self.Rroot: Optional[float] = 0.3
        self.Rtip: Optional[float] = 5.3
        
        self.thetaroot: Optional[float] = 2
        self.thetatwist: Optional[float] = 1

        self.Croot: Optional[float] = 1
        self.Cslope: Optional[float] = -0.1

        self.tailRroot: Optional[float] = None
        self.tailRtip: Optional[float] = None

        self.mass: Optional[float] = 100

class flight_conditions:
    def __init__(self) -> None:
        self.velocity: Optional[float] = 60
        self.rho: Optional[float] = 0.9


class Trim:
    def __init__(self, rotor: Rotor, condition: flight_conditions) -> None:
        self.weight: Optional[float] = 40000
        self.velocity: Optional[float] = 60
        self.rotor = rotor
        self.condition = condition
    
    def get_thrust(self, tail = 0) -> None:
        # get the drag
        drag = 0.5*11.189493432941742*self.condition.rho*(self.velocity)**2
        weight = self.weight

        return np.sqrt((weight)**2 + (drag)**2 + (tail)**2)
    
    def get_Vin(self, Thrust: Optional[float] = None) -> float:
        guess = 2
        def thrust(Vin):
            rho = self.condition.rho
            V = self.condition.velocity
            A = np.pi*(self.rotor.Rtip)**2

            T = Thrust if Thrust is not None else self.get_thrust()
            tpp = self.alpha_tpp()

            return rho*A*(np.sqrt((V*np.cos(tpp)**2) + (V*np.sin(tpp) + Vin)**2))*2*Vin - T
        
        return fsolve(thrust, guess)[0]


    #Getting the beta0

    #alpha tpp  
    def alpha_tpp(self) -> None:
        drag = 0.5*11.189493432941742*self.condition.rho*(self.velocity)**2
        return np.arctan(drag/self.weight)
    
    #pitch - theta
    def get_pitch(self, r,phi,t0: Optional[float] = None, t1c = 0, t1s = 0, b1c: Optional[float] = None, b1s = 0) -> float:
        t0 = t0 if t0 is not None else self.rotor.thetaroot
        b1c = b1c if b1c is not None else self.alpha_tpp()
        c = t0 + (t1c - b1s)*np.cos(phi) + (b1c + t1s)*np.sin(phi)
        return c + r*np.deg2rad(self.rotor.thetatwist)
    
    #get chord length
    def get_chord(self, r) -> float:
        return self.rotor.Croot+ r*self.rotor.Cslope
    
    #alpha
    def get_alpha(self, phi, r, t0, t1c, t1s, b1c, b1s, beta = 0) -> float:
        tpp = self.alpha_tpp()
        Vin = self.get_Vin()
        pitch = self.get_pitch(r, phi, t0, t1c, t1s, b1c, b1s)

        V = self.condition.velocity
        omega = (450*np.pi)/30

        advratio = V*np.cos(tpp)/(omega*(self.rotor.Rtip - self.rotor.Rroot))
        lambag = Vin + V*np.sin(tpp)

        #vin
        vin = (Vin/omega*(self.rotor.Rtip - self.rotor.Rroot))*(1 + ((4*advratio)/(1.2*lambag + advratio)*3)*(r*np.cos(phi))/((self.rotor.Rtip - self.rotor.Rroot)))

        #iteration with beta
        P = V*np.sin(tpp) + vin + V*np.sin(beta)*np.cos(phi)
        T = omega*r + V*np.cos(tpp)*np.sin(phi)

        return pitch - np.arctan(P/T), np.arctan(P/T)
    
    #cl function
    def get_cl(self, phi, r, beta0, t0, t1c, t1s, b1c, b1s):
        
        alpha, kei = self.get_alpha(phi, r, t0, t1c, t1s, b1c, b1s, beta0)
        return 2*np.pi*alpha, alpha, kei
    
    #cd function
    def get_cd(self, cl):
        return 0.01 + 0.02*cl**2
    
    def get_beta(self, t0, t1c, t1s, b1c, b1s, Vin: Optional[float] = None) -> float:
        omega = (450 * np.pi) / 30
        beta0 = 0.0
        phi = np.deg2rad(45)
        I = (self.rotor.mass / (3 * (self.rotor.Rtip - self.rotor.Rroot))) * (
            (self.rotor.Rtip) ** 3 - (self.rotor.Rroot) ** 3
        )
        V = self.condition.velocity

        r = np.linspace(self.rotor.Rroot, self.rotor.Rtip, 10)
        dr = (self.rotor.Rtip - self.rotor.Rroot) / 10

        beta = 1.0
        tol = 0.5  # degrees

        max_iter = 50
        iter_count = 0

        while np.abs(np.rad2deg(beta) - np.rad2deg(beta0)) > tol and iter_count < max_iter:
            beta = beta0
            Vin_val = Vin if Vin is not None else self.get_Vin()
            tpp = self.alpha_tpp()

            beta0 = 0.0
            for k in r:
                c = self.get_chord(k)
                cl = self.get_cl(phi, k, beta0, t0, t1c, t1s, b1c, b1s)[0]

                # velocities
                P = V * np.sin(tpp) + Vin_val + V * np.sin(beta) * np.cos(phi)
                T = omega * k + V * np.cos(tpp) * np.sin(phi)

                beta0 += (
                    0.5 * self.condition.rho * c * (P**2 + T**2) * k * cl * dr
                ) / (np.abs(I) * (omega**2))

            iter_count += 1

            # cap value to avoid invalid arcsin
            beta0 = np.clip(beta0, -1, 1)

        if iter_count >= max_iter:
            print("⚠️ get_beta did not converge within max iterations")

        # return final beta safely
        return np.arcsin(np.clip(beta0, -1, 1))


    
    def main_thrust_trim(self, Vin, beta0, t0, t1c, t1s, b1c, b1s):
        
        tpp = self.alpha_tpp()
        n = 100
        omega = (450*np.pi)/30

        dr = (self.rotor.Rtip - self.rotor.Rroot)/n
        
        V = self.condition.velocity
        advratio = V*np.cos(tpp)/(omega*(self.rotor.Rtip - self.rotor.Rroot))
        lambag = Vin + V*np.sin(tpp)
        
        Thrust = []
        torque = []
        
        phi = np.linspace(0, 90, 4)
        r = np.linspace(self.rotor.Rroot, self.rotor.Rtip, n)
        for i in phi:
            t_blade_phi = 0
            d_blade_phi = 0
            for j in r:
                for k in [i, i + 90, i +180, i + 270]:

                    vin = (Vin/(omega*(self.rotor.Rtip - self.rotor.Rroot)))*(1 + ((4*advratio)/((1.2*lambag + advratio)*3))*((j*np.cos(np.deg2rad(k))))/((self.rotor.Rtip - self.rotor.Rroot)))

                    P = V*np.sin(tpp) + vin + V*np.sin(beta0)*np.cos(np.deg2rad(k))
                    T = omega*j + V*np.cos(tpp)*np.sin(np.deg2rad(k))

                    if T > 0:
                        cl, alpha, kei = self.get_cl(np.deg2rad(k), j, beta0, t0, t1c, t1s, b1c, b1s)
                        cd = self.get_cd(cl)
                        
                        t_blade_phi += (0.5*self.condition.rho*(P**2 + T**2)*(cl*np.cos(kei) - cd*np.sin(kei))*dr)

                        d_blade_phi += (j*0.5*self.condition.rho*(P**2 + T**2)*(cd*np.cos(kei) + cl*np.sin(kei))*dr)

                    
                    elif T < 0:
                        cl= 2*np.pi*self.get_alpha(np.deg2rad(k), j, beta0, t0, t1c, t1s, b1c, b1s)[0]
                        cd = 0.1 + 0.02*cl**2
                        kei = self.get_alpha(np.deg2rad(k), j, beta0, t0, t1c, t1s, b1c, b1s)[1]

                        t_blade_phi += (0.5*self.condition.rho*(P**2 + T**2)*(cl*np.cos(kei) - cd*np.sin(kei))*dr)*np.cos(beta0)

                        d_blade_phi += (j*0.5*self.condition.rho*(P**2 + T**2)*(cd*np.cos(kei) + cl*np.sin(kei))*dr)
            Thrust.append(t_blade_phi)
            torque.append(d_blade_phi)
        
        return np.average(np.array(Thrust)), np.average(np.array(torque))

    def output(self):
        t0  = np.deg2rad(0)
        t1c = np.deg2rad(0)
        t1s = np.deg2rad(0)
        b1s = 0.0
        Ltail = 12.0

        Fx_req = 0.5 * 11.189493432941742 * self.condition.rho * (self.condition.velocity)**2
        Fy_req = self.weight

        tol = 1.0
        max_iter = 100
        iteration_count = 0
        Ttail_prev = 0.0

        T = 40000
        Fx = T * np.sin(self.alpha_tpp())*np.cos(-b1s)
        Fy = T * np.cos(self.alpha_tpp())*np.cos(-b1s)

        print(f"{'Iter':>4} | {'T (N)':>8} | {'Fx':>8} | {'Fy':>8} | {'ResFx':>8} | {'ResFy':>8} | "
            f"{'t0(deg)':>8} | {'t1c(deg)':>10} | {'t1s(deg)':>10} | {'beta(deg)':>10}")

        for _ in range(max_iter):
            iteration_count += 1

            b1c = self.alpha_tpp()
            Vin = self.get_Vin(T)
            beta = self.get_beta(t0, t1c, t1s, b1c, b1s, Vin)
            Ttail = self.main_thrust_trim(Vin, beta, t0, t1c, t1s, b1c, b1s)[1] / Ltail
            T = self.main_thrust_trim(Vin, beta, t0, t1c, t1s, b1c, b1s)[0]
            # Fixed-step update
            t0  += np.deg2rad(0.2) * np.sign(Fy_req - Fy)
            t1c += np.deg2rad(0.2) * np.sign(Fx_req - Fx)
            t1s += np.deg2rad(0.2) * np.sign(Ttail_prev - Ttail)

            Ttail_prev = Ttail

            b1s = -np.arcsin(np.clip(Ttail / max(T, 1e-8), -0.999999, 0.999999))
            Fx = T * np.cos(-b1s) * np.sin(self.alpha_tpp())
            Fy = T * np.cos(-b1s) * np.cos(self.alpha_tpp())
            T = np.sqrt(Fx**2 + Fy**2 + Ttail**2)

            res_fx = Fx - Fx_req
            res_fy = Fy - Fy_req

            print(f"{iteration_count:4d} | {T:8.1f} | {Fx:8.1f} | {Fy:8.1f} | "
                f"{res_fx:8.1f} | {res_fy:8.1f} | "
                f"{np.rad2deg(t0):8.2f} | {np.rad2deg(t1c):10.2f} | {np.rad2deg(t1s):10.2f} | {np.rad2deg(beta):10.2f}")

            if abs(res_fx) < tol and abs(res_fy) < tol:
                print(f"\n✅ Converged in {iteration_count} iterations.")
                break

        Fz = T * np.sin(-b1s)
        Fy = T * np.cos(-b1s) * np.cos(self.alpha_tpp())
        Fx = T * np.cos(-b1s) * np.sin(self.alpha_tpp())

        return (np.rad2deg(t0), np.rad2deg(t1s), np.rad2deg(t1c),
                np.rad2deg(b1s), np.rad2deg(b1c), np.rad2deg(beta),
                Fx, Fy, Fz, np.rad2deg(beta))


In [29]:
rotor = Rotor()
conditions = flight_conditions()

iteration = Trim(rotor, conditions)
iteration.output()

Iter |    T (N) |       Fx |       Fy |    ResFx |    ResFy |  t0(deg) |   t1c(deg) |   t1s(deg) |  beta(deg)
   1 |  71175.4 |  29310.6 |  64678.5 |  11183.7 |  24678.5 |     0.20 |       0.20 |      -0.20 |       7.42
   2 |  74979.2 |  30865.1 |  68108.6 |  12738.1 |  28108.6 |     0.00 |       0.00 |      -0.40 |       8.99
   3 |  68886.8 |  28352.9 |  62565.1 |  10225.9 |  22565.1 |    -0.20 |      -0.20 |      -0.20 |       8.86
   4 |  65547.3 |  26974.3 |  59523.0 |   8847.3 |  19523.0 |    -0.40 |      -0.40 |       0.00 |       8.79
   5 |  62206.7 |  25595.1 |  56479.6 |   7468.1 |  16479.6 |    -0.60 |      -0.60 |       0.20 |       8.72
   6 |  58867.1 |  24216.1 |  53436.6 |   6089.1 |  13436.6 |    -0.80 |      -0.80 |       0.40 |       8.66
   7 |  55529.1 |  22837.4 |  50394.3 |   4710.4 |  10394.3 |    -1.00 |      -1.00 |       0.60 |       8.61
   8 |  52193.3 |  21459.2 |  47353.1 |   3332.2 |   7353.1 |    -1.20 |      -1.20 |       0.80 |       8.56
   9 |  48

KeyboardInterrupt: 

"""
    def output(self):
        # Initial guesses
        t1c, t1s, t0 = np.deg2rad(0.2), np.deg2rad(0.0), np.deg2rad(5.0)
        b1s, Ltail = 0.0, 12.0
        omega = 450 * (np.pi / 30)
        R = self.rotor.Rtip - self.rotor.Rroot

        #for force balance
        Fx = 0.5*10.87*self.condition.rho*(self.velocity)**2
        Fy = self.weight

        tol = 1e-3
        error_fx, error_fy = 1, 1

        Ttail_prev = 0
        iteration_count = 0
        max_iter = 200  
        while ((abs(error_fx) > tol) or (abs(error_fy) > tol)) and iteration_count < max_iter:
            iteration_count += 1
            b1c = self.alpha_tpp()
            Vin = self.get_Vin()
            beta = self.get_beta(t0, t1c, t1s, b1c, b1s)
            Ttail = self.main_thrust_trim(Vin, beta, t0, t1c, t1s, b1c, b1s)[1] / Ltail
            T = self.get_thrust(Ttail)
            b1s = np.arcsin(Ttail / T)

            fx = T * np.cos(b1s) * np.sin(self.alpha_tpp()) - Fx
            fy = T * np.cos(b1s) * np.cos(self.alpha_tpp()) - Fy

            if fx > tol:
                t1c += np.deg2rad(0.05)
            elif fx < -tol:
                t1c -= np.deg2rad(0.04)

            if fy > tol:
                t0 += np.deg2rad(0.05)
            elif fy < -tol:
                t0 -= np.deg2rad(0.04)
            
            if Ttail > Ttail_prev:
                t1s -= np.deg2rad(0.05)
            elif Ttail < Ttail_prev:
                t1s += np.deg2rad(0.04)

            Ttail_prev = Ttail
            error_fx, error_fy = fx, fy

        beta0 = beta
        Fz = T * np.sin(b1s)
        Fy = T * np.cos(b1s) * np.cos(self.alpha_tpp())
        Fx = T * np.cos(b1s) * np.sin(self.alpha_tpp())

        return np.rad2deg(t0), np.rad2deg(t1s), np.rad2deg(t1c), np.rad2deg(b1s), np.rad2deg(b1c), np.rad2deg(beta0), Fx, Fy, Fz
"""

"""

    def output(self):
        # Initial guesses (radians)
        t1c, t1s, t0 = np.deg2rad(0.1), np.deg2rad(0.0), np.deg2rad(2.0)
        b1s, Ltail = 0.0, 12.0
        omega = 450 * (np.pi / 30)
        R = self.rotor.Rtip - self.rotor.Rroot

        # force targets
        Fx_req = 0.5 * 11.189493432941742 * self.condition.rho * (self.condition.velocity)**2
        Fy_req = self.weight

        tol = 0.1
        error_fx, error_fy = 1e9, 1e9

        Ttail_prev = 0.0
        iteration_count = 0
        max_iter = 500


        while ((abs(error_fx) > tol) or (abs(error_fy) > tol)) and iteration_count < max_iter:
            iteration_count += 1

            b1c = self.alpha_tpp()
            Vin = self.get_Vin()                           
            beta = self.get_beta(t0, t1c, t1s, b1c, b1s)  
            Ttail = self.main_thrust_trim(Vin, beta, t0, t1c, t1s, b1c, b1s)[1] / Ltail
            T = self.get_thrust(Ttail)

            b1s = -np.arcsin(Ttail / T)

            fx = T * np.cos(-b1s) * np.sin(self.alpha_tpp()) - Fx_req
            fy = T * np.cos(-b1s) * np.cos(self.alpha_tpp()) - Fy_req

            relax = 0.7

            if abs(fx) > tol:
                t1c += relax * np.sign(fx) * np.deg2rad(0.05)

            if abs(fy) > tol:
                t0 += relax * np.sign(fy) * np.deg2rad(0.05)

            if Ttail > Ttail_prev:
                t1s -= np.deg2rad(0.5)
            elif Ttail < Ttail_prev:
                t1s += np.deg2rad(0.4)
            

            Ttail_prev = Ttail
            error_fx, error_fy = fx, fy

        beta0 = beta
        Fz = T * np.sin(-b1s)
        Fy = T * np.cos(-b1s) * np.cos(self.alpha_tpp())
        Fx = T * np.cos(-b1s) * np.sin(self.alpha_tpp())

        return np.rad2deg(t0), np.rad2deg(t1s), np.rad2deg(t1c), np.rad2deg(b1s), np.rad2deg(b1c), np.rad2deg(beta0), Fx, Fy, Fz
"""